<a href="https://colab.research.google.com/github/Lifeisforty2/Lifeisforty2.github.io/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
///////////////****************FIXME cell****************////////////////////////

    Here we use PyDictionary to compare user answers to the trained answers.
    Since there are many ways a question can be answered, it can be assumed the
    user may provide an answer that differs from the expected answer.
    Thus, if the bot deems the user answer to be 'incorrect', it will then check
    if the answer is 'close enough' to th expected answer.
    This will be achieved by comparing words used in the expected and given answers
    and check if the expected word and given word:
        - are synonyms
        - have similar meanings
        - share a synonym or synonyms
    The results of these checks will provide an overall accuracy of the given answer.
    This accuracy will determine if the user answer is 'correct'.
'''
from numpy.ma import empty
import numpy as np
from PyDictionary import PyDictionary
dictionary = PyDictionary() # create a new english dictionary

'''
    def get_answer_accuracy(answer) determines if the user's answer is 'correct'
    or accurate enough to the expected answer.
'''
def get_answer_accuracy(user_response, question_asked):
  answer_acc = 0.0   # initialize to a float
  valid_word = False # used to check if given word is a valid word
  correct = 0
  total = 0

  # split each response into a numpy array of words
  user_words = np.char.split(user_response)
  expected_words = np.char.split(question_asked)

  # loop through the arrays and compare each word
  for word in enumerate(user_words):
    for expected in enumerate(question_asked):
      # check if the word is a valid word
      valid_word = bool(dictionary.meaning(word))

      if (valid_word == True):
        # if the words match exactly
        if word == expected:
          correct += 1
          total += 1

        # if the words do not match
        else:
          # check if the words are synonyms
          synonyms = dictionary.synonym(expected)

          if word in synonyms:
            correct += 1
            total += 1

          # if the words are not synonyms, check if they share a synonym
          else:
            synonyms2 = dictionary.synonym(word)

            # check if the intersection of the synonym lists contain any shared elements
            shared = bool(set(synonyms).intersection(synonyms2))
            if (len(shared) > 0 and word not in synonyms and expected not in synonyms2 and shared == True):
              correct += 1
              total += 1

           # if the words don't share a synonym
            else:
              total += 1

  # get the overall accuracy
  answer_acc = correct / total

  return answer_acc   # return the overall accuracy of the user's answer

ModuleNotFoundError: ignored

In [2]:
# create labels list with index labels
# create questions list with questions
labels = [1, 2, 3, 4, 4]
questions = ["define a class", "define a function", "define a loop", "define a class variable in java", "define a instance variable in java"]
# answers list with answers
#class var and instanc var have same answer
answers = ["a class is a blueprint for the object", "a function is a block of code to carry out a specific task", "a loop is used to iterate over a sequence of elements", \
           "a class variable (sometimes called an instance variable) is a variable that is associated with a class and not objects of that class. It is a variable that belongs to the class and not to object(instance)."]

In [3]:
'''
    Here we will use the CountVectorizer class from sklearn to convert the questions to vectors
    The CountVectorizer class converts a collection of text documents to a matrix of token counts
    Each row represents a document in the corpus, and each column represents a token in the vocabulary
    The value in each cell is the frequency of occurrence of a particular token in a particular document
    The CountVectorizer class has a fit_transform() method that converts the questions to vectors
    The fit_transform() method takes the questions as input and returns a matrix of token counts
    The fit_transform() method also creates a vocabulary of unique tokens from the questions
    The vocabulary is a dictionary that maps each token to a unique integer index

'''
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer() # create an instance of CountVectorizer
training_centers = vectorizer.fit_transform(questions) # fit_transform() method to convert the questions to vectors


In [4]:
'''
    Here we will use the MultinomialNB class from sklearn to create a Naive Bayes classifier
'''
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB() # create our classifier
classifier.fit(training_centers, labels) # train our classifier


MultinomialNB()

In [6]:
global score # keeps number of correct/incorrect
score = {"correct": 0, "incorrect": 0}
global questions_asked # keeps track of questions asked to exit chat
questions_asked = []

import random
class ChatBot:
    exit_commands = ["quit", "pause", "exit", "goodbye", "bye", "later", "stop"]
    '''
        def start_chat(self) starts the chat by asking the user a question
    '''
    def start_chat(self):
        # introduce the chatbot and ask if the user is ready to begin the interview
        print("Hi, I'm a chatbot trained to ask you questions about data science\n")
        print("During the interview, you will be asked a short series of programming questions\n")
        print("that focus on Java and general logic functions.\n")
        print("If at any time you wish to exit the interview, type 'quit', 'exit', 'goodbye', 'bye', or 'stop'.\n")
        print("Note that all progress will be lost if you exit before the interview concludes!\n")
        user_response = input("Would you like to begin the interview?\n")
        # if the user is ready to begin the interview
        if self.make_exit(user_response) == False:
            # respond with a great!
            print("Great! Let's begin.\n")

            # ask the user a question from the list of questions
            aiquestion = self.generate_question()
            # pass the question to the chat method which will take the input from the user and check if it's correct
            self.chat(aiquestion)
        # if the user is not ready to begin the interview
        else:
            # end the chat
            self.exit_chat()

    '''
        def chat(self, questionAsked) takes in a question and asks the user the question
        then takes the user's input and checks if it's correct

    '''
    def chat(self, questionAsked):
        user_response = input(questionAsked + "\n") # ask the user the question and get the response

        while not self.make_exit(user_response): # while the user doesn't want to exit
            # if questions_asked is full then end the chat
            # append the question asked to the list of questions asked
            questions_asked.append(questionAsked) # FIXME: uncomment this line

            if self.check_answer(user_response, questionAsked): # check if the answer is correct
                score["correct"] += 1
                print("Correct!")
                if len(questions_asked) == len(questions):
                    print("That's all the questions I have for you today. But before you go, let's see how you did.")
                    print("You got " + str(score["correct"]) + " questions correct and " + str(score["incorrect"]) + " questions incorrect.")
                    return
                questionAsked = self.generate_question() # generate another question
                user_response = input(questionAsked + "\n") # ask the user the question and get the response
            else: # if the answer is incorrect, then pick another question
                score["incorrect"] += 1 # increment the number of incorrect answers FIXME: uncomment this line
                print("Incorrect!")
                if len(questions_asked) == len(questions):
                    print("That's all the questions I have for you today. But before you go, let's see how you did.")
                    print("You got " + str(score["correct"]) + " questions correct and " + str(score["incorrect"]) + " questions incorrect.")
                    return
                questionAsked = self.generate_question() # generate another question
                user_response = input(questionAsked + "\n") # ask the user the question and get the response
        return
    '''
        def check_answer(self, user_response, questionAsked) takes in the user's response and the question asked
        and returns True if the answer is correct and False otherwise
        this is done by first predicting the label of the question asked and then using the label to get the answer
        then the answer is compared to the user's response
    '''
    def check_answer(self, user_answer, questionAsked):
        # predict the label of the question so we can get the index of the question
        input_vector = vectorizer.transform([questionAsked]) # convert the question to a vector
        predict = classifier.predict(input_vector) # predict the label of the vector
        index = int(predict[0]) # get the index of the label
        print("Accuracy of the classifiers prediction: " + str(classifier.predict_proba(input_vector)[0][index-1]*100)[:5] + "%") # FIXME: print the accuracy of the prediction
        # use the index to get the answer from the answers list
        correct_answer = answers[index-1]

        # remove punctuation and convert to lowercase
        # remove parenthesis and commas
        for char in '-.,\n()':
            correct_answer = correct_answer.replace(char,' ')
            user_answer = user_answer.replace(char,' ')

        ##### To increase accuracy, i created two list of common short forms of words and a list with the correct form of the words #####
        # sometimes people use short forms of words like "that's" instead of "that is" or "it's" instead of "it is"
        # replace the user's answer with the correct word
        short_forms = ["that's", "it's", "i'm", "you're", "we're", "they're"]
        short_forms_without_apostrophe = ["thats", "its", "im", "youre", "were", "theyre"]
        correct_form = ["that is", "it is", "i am", "you are", "we are", "they are"]
        for i in range(len(short_forms)):
            if short_forms[i] in user_answer:
                user_answer = user_answer.replace(short_forms[i], correct_form[i])
            elif short_forms_without_apostrophe[i] in user_answer:
                user_answer = user_answer.replace(short_forms_without_apostrophe[i], correct_form[i])

        correct_answer = correct_answer.strip() # remove extra spaces
        user_answer = user_answer.strip()       # remove extra spaces
        correct_answer = correct_answer.lower().split() # convert to lowercase and split into a list
        user_answer = user_answer.lower().split()       # convert to lowercase and split into a list

        common = set(correct_answer).intersection(user_answer) # get the words in common
        # show accuracy in the users answer
        # accuracy = number of words in common/number of words in correct answer
        accuracy = len(common)/len(correct_answer)
        print("Accuracy of users answer: " + str(accuracy*100)[:5] + "%")
        #print("correct_answer: " + str(correct_answer))
        #print("user_answer: " + str(user_answer))
        # if the accuracy is greater than 45% then the answer is correct
        if accuracy > 0.45:
            return True # the answer is correct
        else:
            return False # the answer is incorrect

    '''
        def make_exit(self, reply) takes in a reply and returns True if the user wants to exit
        and False otherwise
    '''
    def make_exit(self, reply):
        for exit_command in self.exit_commands:
            if exit_command in reply:
                print("Goodbye!")
                return True
        return False
    '''
        def generate_question(self) returns a random question from the list of questions
    '''
    def generate_question(self):
        # create a list of questions asked to keep track of which questions have been asked
        # and remove the question from the list of questions
        selected_question = random.choice(questions) # select a random question from the list of questions
        # if the question has already been asked then select another random question
        while selected_question in questions_asked:
            selected_question = random.choice(questions) # select another random question from the list of questions
        return selected_question



In [ ]:
etcetra = ChatBot()
etcetra.start_chat()

Hi, I'm a chatbot trained to ask you questions about data science

During the interview, you will be asked a short series of programming questions

that focus on Java and general logic functions.

If at any time you wish to exit the interview, type 'quit', 'exit', 'goodbye', 'bye', or 'stop'.

Note that all progress will be lost if you exit before the interview concludes!

Would you like to begin the interview?
yes
Great! Let's begin.

define a loop
a loop is a blueprint for the object
Accuracy of the classifiers prediction: 40.60%
Accuracy of users answer: 27.27%
Incorrect!
define a function
a function is a block of code that carries out a specific task
Accuracy of the classifiers prediction: 40.60%
Accuracy of users answer: 69.23%
Correct!
define a class
a class is a blueprint for the object
Accuracy of the classifiers prediction: 34.17%
Accuracy of users answer: 87.5%
Correct!
define a instance variable in java
an instance variable, sometimes called a class variable is one that is a